## Welcome to Agentic AI - Day 2 foundations

In [ ]:
# Start with imports - ask ChatGPT to explain any package that has some confusions.

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display


In [ ]:
# Always remember to load env variables.
load_dotenv(override=True)

In [ ]:
# Print the key prefixes to help with Debugging

openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")


if openai_api_key:
    print(f"OpenAI API Key is {openai_api_key[:8]}")
else:
    print("OpenAI API Key is not set")

if anthropic_api_key:
    print(f"Anthropic API Key is {anthropic_api_key[:8]}")
else:
    print("Anthropic API Key is not set")

if google_api_key:
    print(f"Google API Key is {google_api_key[:8]}")
else:
    print(f"Google API Key is not set")

if deepseek_api_key:
    print(f"Deepseek API Key is {deepseek_api_key[:8]}")
else:
    print("Deepseek API Key is not set")

if groq_api_key:
    print(f"Groq API Key is {groq_api_key[:8]}")
else:
    print("Groq API Key is not set")

In [ ]:
request = "Please come up with a challenging, nuanced questions that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation. "
messages = [{"role":"user", "content": request}]

In [ ]:
messages

In [ ]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-5-mini",
    messages=messages,
)

question = response.choices[0].message.content
print(question)

In [ ]:
competitors = []
answers = []
messages = [{"role":"user", "content": request}]

In [ ]:
# The API we know well

model_name = "gpt-5-mini"
response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# Anthropic has a slightly different API, and Max Tokens is required.

model_name = "claude-3-7-sonnet-latest"


claude = Anthropic()
response = claude.messages.create(
    model=model_name,
    messages=messages,
    max_tokens=1000
)

answer = response.choices[0].text

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


In [ ]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
!ollama pull llama3.2

In [ ]:
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# so where are we?

print(competitors)
print(answers)

In [ ]:
# it's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")

In [ ]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from Competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

In [ ]:
judge = f"""You are judging a competition between {len(competitors)} competitors,
Each model has been given this question:

{question}

Your Job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", .....]}}

Here are the responses from each competitor:

{together}

Now Respond with the JSON with the ranked order of the competitors, nothing else. DO not include markdown formatting or code blocks
"""


In [ ]:
print(judge)

In [ ]:
judge_messages = [{"role": "user", "content": judge}]

In [ ]:
# Judgement time!

openai = OpenAI()

response = openai.chat.completions.create(
    model="gpt-5-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)

In [ ]:
# Ok let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")